In [54]:
# !pip install catboost

In [9]:
import joblib
import numpy as np
import pandas as pd
import catboost
print(catboost.__version__)
import sklearn

print(sklearn.__version__)
from catboost import CatBoostClassifier
from datetime import datetime, timedelta

from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import permutation_importance
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split, cross_val_score

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

rs = 0

1.0.6
1.0.2


In [16]:
today = datetime.now().today().time().strftime('%H:%M:%S')
str(today)

'09:58:46'

# Dataset

In [ ]:
today = 

In [2]:
df = pd.read_feather('sampling.feather')

# Missing

In [57]:
missing_limit = 400000
missing_count = df.isna().sum().sort_values(ascending=False).to_frame().reset_index()
display(missing_count.describe())

,0
count,191.000000
mean,152209.000000
std,318579.970109
min,0.000000
25%,0.000000
50%,1231.000000
75%,31519.000000
max,999251.000000


In [58]:
df = df[missing_count.loc[missing_count[0] < 400000]['index'].to_list()]
df.shape

(1000000, 160)

In [59]:
df.select_dtypes(exclude=["number","bool_"])
df = df.drop(['customer_ID', 'S_2'], axis=1)

In [60]:
# enc = OrdinalEncoder()
# ordinal_encode_D_63 = enc.fit_transform(df.select_dtypes(exclude=["number","bool_"]))
# ordinal_encode_D_63 = pd.DataFrame(ordinal_encode_D_63, columns=['D_63'])
# df['D_63'] = ordinal_encode_D_63.D_63.to_list()

In [61]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis = 1), df[['target']], test_size=0.25, random_state=rs)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(750000, 157)
(250000, 157)
(750000, 1)
(250000, 1)


In [62]:
cat_features = list(df.select_dtypes(exclude=["number","bool_"]).columns)
cat_features

['D_64', 'D_63']

In [63]:
X_train.loc[:, cat_features] = X_train.loc[:, cat_features].astype(str)

# Modeling

In [64]:
catboost_train_x, catboost_valid_x, catboost_train_y, catboost_valid_y = train_test_split(X_train, y_train, test_size=0.25, random_state=rs)

In [65]:
catboost_train_x

,D_43,S_27,D_46,S_3,S_7,D_62,D_48,D_61,P_3,D_78,...,B_28,R_11,R_10,S_16,R_8,R_7,B_24,D_75,P_4,B_23
502202,0.059735,0.454198,NaN,0.069483,0.052458,0.394483,0.001024,0.234478,0.646616,0.005498,...,0.036401,0.003062,0.008365,0.009597,0.003773,0.003960,0.000065,0.072344,0.002308,0.027627
656964,0.038633,0.494255,0.405865,0.139724,0.108579,0.034046,0.529142,0.176623,-0.018035,0.004799,...,0.108571,0.005739,0.009965,0.003566,0.000866,4.590338,0.009972,0.337809,0.008656,0.075300
25952,0.239731,0.266298,0.393734,0.108152,0.077640,0.120995,0.172997,0.601090,0.556809,0.003748,...,0.254329,0.007290,0.004201,0.005898,0.006207,0.009587,0.003328,0.205489,0.007560,0.192428
500038,0.166910,0.004575,0.443789,0.504020,0.445658,0.008733,0.639058,0.729508,0.509179,0.002963,...,0.079094,0.000598,0.009353,0.009928,0.006647,0.004493,0.008854,0.202941,0.957248,0.232632
329490,0.041838,NaN,0.504826,0.446329,0.344703,0.039932,0.941534,0.903026,0.695111,0.000353,...,0.219110,0.003296,0.008363,0.003048,0.001955,0.007762,0.005480,0.271459,0.005760,0.485515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941404,0.024581,0.275628,0.437053,0.130866,0.093882,NaN,0.002190,NaN,0.489444,0.000104,...,0.115589,0.006328,0.008995,0.009303,0.006175,0.003858,0.007433,0.006851,0.001079,0.020035
890776,0.038985,0.000020,0.444156,0.204615,0.169270,0.064431,0.066159,0.021294,0.636985,0.002730,...,0.012626,0.008352,0.008158,0.004195,0.006924,0.006911,0.003126,0.068623,0.004584,0.013922
343719,0.064038,0.006749,0.478148,0.149951,0.102899,0.255739,0.395968,0.786577,0.709838,0.007268,...,0.893792,0.503240,0.004057,0.002166,0.000464,0.009931,0.000852,0.068859,0.002881,0.045450
43480,NaN,NaN,0.454045,NaN,NaN,0.152854,0.457047,0.649644,0.553035,0.001341,...,0.123576,0.005918,0.009338,0.003784,0.009951,0.003435,0.006145,0.134540,0.007459,0.224577


In [66]:
params = {'loss_function':'CrossEntropy', # objective function
          'eval_metric':'Accuracy', # metric
          'cat_features': cat_features,
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': 42,
          'iterations': 3000,
          'l2_leaf_reg': 5,
          
         }
cbc = CatBoostClassifier(**params)
cbc.fit(catboost_train_x, catboost_train_y, # data to train on (required parameters, unless we provide X as a pool object, will be shown below)
          eval_set=(catboost_valid_x, catboost_valid_y), # data to validate on
          use_best_model=True, # True if we don't want to save trees created after iteration with the best validation score
          plot=True # True for visualization of the training process (it is not shown in a published kernel - try executing this code)
         )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8466009	test: 0.8471680	best: 0.8471680 (0)	total: 483ms	remaining: 24m 9s
200:	learn: 0.8718436	test: 0.8710507	best: 0.8710507 (198)	total: 43.5s	remaining: 10m 6s
400:	learn: 0.8759022	test: 0.8748213	best: 0.8748213 (400)	total: 1m 25s	remaining: 9m 15s
600:	learn: 0.8780551	test: 0.8765600	best: 0.8765867 (599)	total: 2m 20s	remaining: 9m 19s
800:	learn: 0.8799484	test: 0.8777547	best: 0.8778133 (788)	total: 3m 5s	remaining: 8m 30s
1000:	learn: 0.8814880	test: 0.8785760	best: 0.8785760 (1000)	total: 3m 53s	remaining: 7m 46s
1200:	learn: 0.8828462	test: 0.8789707	best: 0.8790507 (1184)	total: 4m 42s	remaining: 7m 2s
1400:	learn: 0.8840853	test: 0.8793973	best: 0.8794507 (1395)	total: 5m 31s	remaining: 6m 18s
1600:	learn: 0.8852284	test: 0.8796427	best: 0.8797013 (1588)	total: 6m 20s	remaining: 5m 32s
1800:	learn: 0.8863360	test: 0.8799413	best: 0.8800053 (1775)	total: 7m 8s	remaining: 4m 45s
2000:	learn: 0.8873547	test: 0.8802133	best: 0.8803520 (1966)	total: 7m 58s	rem

In [50]:
joblib.dump(pipeline, "./random_forest.joblib")

['./random_forest.joblib']

In [47]:
tf_test = pipeline['imputer'].transform(X_test)
pipeline['rfc'].predict(tf_test)

array([1, 0, 0, ..., 1, 0, 1])

In [51]:
loaded_rf = joblib.load("./random_forest.joblib")

In [ ]:
loaded_rf['rfc']

RandomForestClassifier(random_state=0)

In [53]:
f1_score(y_test, pipeline['rfc'].predict(tf_test), average='weighted')

0.8744597140527057